In [35]:
import h5py
import tables
import numpy as np
import pandas as pd
import sys
import os
import bz2
import math
import arkouda as ak
import timeit
from glob import glob
from decimal import Decimal

In [2]:
ak.connect(server='localhost',port='5555')

4.3.2
psp =  tcp://localhost:5555
connected to tcp://localhost:5555


In [30]:
#print(os.getcwd())
data = bz2.open("../../data/bz2/netflow_day-05.bz2")
df = pd.read_csv(data)
#Following line writes over the first row of data
df.columns=['Time','Duration','SrcDevice','DstDevice','Protocol','SrcPort','DstPort','SrcPackets','DstPackets','SrcBytes','DstBytes']
df.head()

,Time,Duration,SrcDevice,DstDevice,Protocol,SrcPort,DstPort,SrcPackets,DstPackets,SrcBytes,DstBytes
0,345600,0,Comp950882,Comp387111,17,Port51715,53,1,0,64,0
1,345600,0,Comp158520,Comp576031,17,Port29380,514,5,0,3272,0
2,345600,0,Comp523808,Comp343771,17,Port62088,53,1,1,59,59
3,345600,0,Comp523808,Comp705605,17,Port62088,53,1,1,59,59
4,345600,0,Comp523808,Comp225307,17,Port62088,53,1,1,59,59


In [31]:
millnames = ['',' Thousand',' Million', ' Billion', ' Trillion', 'Quadrillion']
def millify(n):
    n=float(n)
    millidx=max(0,min(len(millnames)-1,
                     int(math.floor(0 if n == 0 else math.log10(abs(n))/3))))
    return '{:.0f}{}'.format(n/10**(3*millidx), millnames[millidx])

In [32]:
#df = pd.DataFrame(data.values,columns=['Time','Duration','SrcDevice','DstDevice','Protocol','SrcPort','DstPort','SrcPackets','DstPackets','SrcBytes','DstBytes'])
#df = pd.DataFrame(data.values, columns = [list(range(11))])
rows, columns = df.shape
#rows = '%.2E' % Decimal(rows)
print(str(millify(rows))+', '+ str(columns))

199 Million, 11


In [33]:
df['SrcPort'].to_hdf('../../data/hdf/SrcPort-netflow_day-05.h5',key='df', mode='w')

In [53]:
#Series Dataframe
df_Src=pd.read_hdf('../../data/hdf/SrcPort-netflow_day-04.h5', 'df')
print(df_Src.shape)
df_Src.head(5)
#print(str(millify(df_Src.shape[0]))+', '+ str(df_Src.shape[1]))

(222323502,)


0    Port46266
1    Port77381
2    Port67870
3    Port49568
4    Port49568
Name: SrcPort, dtype: object

In [36]:
hdf = '../../data/hdf'

In [50]:
print(os.path.join(hdf, 'SrcPort*.h5'))
print(hdf)
hdfFiles=glob(os.path.join(hdf, 'SrcPort*.h5'))
print(type(hdfFiles))
print(hdfFiles)

../../data/hdf/SrcPort*.h5
../../data/hdf
<class 'list'>
['../../data/hdf/SrcPort-netflow_day-04.h5', '../../data/hdf/SrcPort-netflow_day-05.h5', '../../data/hdf/SrcPort-netflow_day-03.h5']


In [ ]:
Columns = ['srcIP','dstIP',...]
len(allfiles)

In [56]:
#file objext, acts like python dictionary, thus we can check keys -> there is one dataset 'df'
import h5py
f = h5py.File('../../data/hdf/SrcPort-netflow_day-04.h5','r')
list(f.keys())

['df']

In [57]:
data = ak.read_hdf('df',hdfFiles)

RuntimeError: Error: readhdf: Rank 0 arrays not implemented

In [ ]:
nrecords=data['srcIP'].size